# A1 Manager Playbook

This notebook demonstrates how to load the **A1_manager** project as a module and execute its test suite programmatically.
            

In [ ]:
# Ensure the project module path is set
import sys
from pathlib import Path

# Insert the 'a1_manager' directory into sys.path
project_root = Path.cwd()
sys.path.insert(0, str(project_root / 'a1_manager'))
print('Module path set to:', project_root / 'a1_manager')

Module path set to: c:\repos\A1_manager\a1_manager


In [ ]:
# Import key classes to verify module loading
from microscope_hardware.nikon import NikonTi2
from a1_manager.main import A1Manager

print('Imported classes successfully:')
print('NikonTi2:', NikonTi2)
print('A1Manager:', A1Manager)

Imported classes successfully:
NikonTi2: <class 'microscope_hardware.nikon.NikonTi2'>
A1Manager: <class 'a1_manager.main.A1Manager'>


In [13]:
# Test NikonTi2
from unittest.mock import MagicMock
from microscope_hardware.nikon import NikonTi2

# Stub Core
core = MagicMock()
# Valid objective
nikon = NikonTi2(core, objective="10x", focus_device="PFSOffset")
# Should have called set_property twice: one for focus, one for FocusMaintenance
core.set_property.assert_any_call('Nosepiece', 'State', 1)
core.set_property.assert_any_call('PFS', 'FocusMaintenance', 'On')

# Invalid objective
try:
    NikonTi2(core, objective="30x")
    raise AssertionError("Expected ValueError for unsupported objective")
except ValueError:
    pass

# Changing focus device toggles FocusMaintenance
nikon.select_focus_device("ZDrive")
core.set_property.assert_any_call('Core', 'Focus', 'ZDrive')
core.set_property.assert_any_call('PFS', 'FocusMaintenance', 'Off')

print("✅ NikonTi2 high-level tests passed.")

✅ NikonTi2 high-level tests passed.


In [18]:
# Test AndorCamera
from unittest.mock import MagicMock
from microscope_hardware.cameras import AndorCamera

core = MagicMock()
# Initialize with binning=4, exposure=123
cam = AndorCamera(core, binning=4, exposure_ms=123)
core.set_property.assert_any_call('LightPath', 'State', 1)
core.set_property.assert_any_call('Andor sCMOS Camera', 'Binning', '4x4')
core.set_property.assert_any_call('Andor sCMOS Camera', 'Exposure', 123)
assert cam.binning == 4 and cam.exposure_ms == 123

# Change settings after init
cam._set_camera_binning(2)
core.set_property.assert_any_call('Andor sCMOS Camera', 'Binning', '2x2')
cam.set_camera_exposure(200)
core.set_property.assert_any_call('Andor sCMOS Camera', 'Exposure', 200)

print("✅ AndorCamera high-level tests passed.")

✅ AndorCamera high-level tests passed.


In [15]:
# Test get_lamp factory
from microscope_hardware.lamps_factory import get_lamp
from microscope_hardware.lamps.pe800 import pE800
from microscope_hardware.lamps.pe4000 import pE4000
from microscope_hardware.lamps.dia_lamp import DiaLamp

core = MagicMock()

lamp = get_lamp(core, 'pE-800')
assert isinstance(lamp, pE800)
lamp = get_lamp(core, 'pE-4000')
assert isinstance(lamp, pE4000)
lamp = get_lamp(core, 'DiaLamp')
assert isinstance(lamp, DiaLamp)

try:
    get_lamp(core, 'UnknownLamp')
    raise AssertionError("Expected ValueError for invalid lamp name")
except ValueError:
    pass

print("✅ get_lamp factory tests passed.")

✅ get_lamp factory tests passed.


In [20]:
# Test Dmd class basics
from unittest.mock import MagicMock
import numpy as np
from microscope_hardware.dmd_manager import Dmd

# Stub Core so all methods exist
core = MagicMock()
core.get_slm_width.return_value = 8
core.get_slm_height.return_value = 6

dmd = Dmd(core, trigger_mode="InternalExpose")
# Should have loaded fullON mask on init
core.display_slm_image.assert_called_once()
# Switch trigger mode
dmd._set_trigger_mode("ExternalBulb")
core.set_property.assert_any_call('Mosaic3', 'TriggerMode', "ExternalBulb")
# Project a custom mask
mask = np.zeros((6,8), dtype='uint8')
dmd._project_mask(mask)
core.set_slm_image.assert_called_with('Mosaic3', mask)
core.display_slm_image.assert_called()
print("✅ Dmd high-level tests passed.")

2025-04-24 15:39:45,728 [WARNING] No transfo_matrix found.


✅ Dmd high-level tests passed.


In [25]:
# Test A1Manager size + oc_settings logic
from unittest.mock import MagicMock
import logging
from a1_manager.main import A1Manager, OPTICAL_CONFIGURATION

# Stub Core and hardware classes
core = MagicMock()
# Monkey-patch Nikon, Camera, Lamp, Dmd inside A1Manager module
import a1_manager.main as M
M.NikonTi2 = lambda core, obj, fd: type("N",(),{"objective":obj, "focus_device":fd, "set_light_path":lambda self,x: setattr(self,"lp",x)})()
M.AndorCamera = lambda core, binning, exp: type("C",(),{"binning":binning,"exposure_ms":exp,"set_camera_exposure":lambda self,e: setattr(self,"exposure_ms",e)})()
M.get_lamp      = lambda core, name: type("L",(),{"lapp_main_branch":None,"preset_channel":lambda self,oc,i: setattr(self,"oc",(oc,i))})()
M.Dmd           = lambda core, mode: type("D",(),{"set_dmd_exposure":lambda self,s:setattr(self,"exp",s),"activate":lambda self:None,"load_dmd_mask":lambda self,m,t:m})()

mgr = A1Manager("10x", exposure_ms=50, binning=1, lamp_name="pE-800")
# Test pixel→µm conversion
assert abs(mgr._size_pixel2micron(100) - (100*0.6461*1))<1e-6
assert mgr.image_size == (2048//1,2048//1)

# Prepare a fake GFP entry with a custom light_path
cfg = OPTICAL_CONFIGURATION.copy()
cfg['light_path'] = 2
cfg['TEST'] = {**cfg['GFP']}
mgr_oc = mgr
# Apply oc_settings with default light_path
mgr_oc.oc_settings('GFP')
assert mgr.camera.exposure_ms == OPTICAL_CONFIGURATION['GFP']['exposure_ms']
assert mgr.nikon.lp == OPTICAL_CONFIGURATION['light_path']

# Override light_path via param
mgr_oc.oc_settings('GFP', light_path=0)
assert mgr.nikon.lp == 0

print("✅ A1Manager high-level tests passed.")

✅ A1Manager high-level tests passed.


In [4]:
# StageCoord tests
from utils.utility_classes import StageCoord

sc = StageCoord(xy=(10, 20), ZDrive=5, PFSOffset=7)
assert sc["xy"] == (10, 20)
assert sc["ZDrive"] == 5
assert sc["PFSOffset"] == 7

sc_copy = sc.copy()
assert isinstance(sc_copy, StageCoord)
assert sc_copy.xy == sc.xy
print("✅ StageCoord tests passed.")

✅ StageCoord tests passed.


In [5]:
#Dataclass JSON encode/decode
from utils.json_utils import encode_dataclass, decode_dataclass
from utils.utility_classes import WellCircleCoord

wc = WellCircleCoord(center=(1, 2), radius=3, ZDrive=4, PFSOffset=5)
encoded = encode_dataclass(wc)
assert encoded["__class__"] == "WellCircleCoord"
decoded = decode_dataclass(encoded)
assert isinstance(decoded, WellCircleCoord)
assert decoded.center == wc.center and decoded.radius == wc.radius
print("✅ Dataclass encode/decode passed.")

✅ Dataclass encode/decode passed.


In [6]:
# bounding_box_nDim
import numpy as np
from utils.utils import bounding_box_nDim

mask = np.array([[0,0,0],
                 [0,1,0],
                 [0,0,0]], dtype=bool)
sub, sl = bounding_box_nDim(mask)
assert sub.shape == (1,1)
assert sl == (slice(1,2), slice(1,2))
print("✅ bounding_box_nDim passed.")

✅ bounding_box_nDim passed.


In [7]:
# draw_square_from_circle
from utils.utils import draw_square_from_circle

sub, sl = draw_square_from_circle((1,1), radius=1, mask_size=(3,3))
# The bounding box should cover at least the center pixel
assert sub.size >= 1
print("✅ draw_square_from_circle passed. bbox slices:", sl)

✅ draw_square_from_circle passed. bbox slices: (slice(1, 2, None), slice(1, 2, None))


In [8]:
# threshold_img
import numpy as np
from utils.utils import threshold_img

img = np.array([[0, 50],[100, 150]], dtype=np.uint16)
mask = threshold_img(img)
assert mask.dtype == bool and mask.shape == img.shape
print("✅ threshold_img passed.")

✅ threshold_img passed.


In [9]:
# _normalize99
import numpy as np
from utils.utils import _normalize99

arr = np.array([0.0, 50.0, 100.0], dtype=float)
norm = _normalize99(arr, lower=0, upper=100)
assert abs(norm[0] - 0.0) < 1e-6
assert abs(norm[-1] - 1.0) < 1e-6
print("✅ _normalize99 passed.")

✅ _normalize99 passed.


In [10]:
# find_circle
from dish_manager.dish_utils.geometry_utils import find_circle

c, r = find_circle((1,0),(0,1),(-1,0))
assert abs(c[0]) < 1e-6 and abs(c[1]) < 1e-6
assert abs(r - 1.0) < 1e-6
print("✅ find_circle passed. center:", c, "radius:", r)

✅ find_circle passed. center: (0.0, 0.0) radius: 1.0


In [11]:
# compute_optimal_overlap
from dish_manager.dish_utils.geometry_utils import compute_optimal_overlap

ov = compute_optimal_overlap((10.0,20.0), well_width=5.0, well_length=10.0)
assert isinstance(ov, tuple) and len(ov) == 2
print("✅ compute_optimal_overlap passed. overlap:", ov)

✅ compute_optimal_overlap passed. overlap: (0.0, 0.5)


In [26]:
# randomise_fov
from dish_manager.dish_utils.geometry_utils import randomise_fov
from utils.utility_classes import StageCoord

grid = {i: StageCoord(xy=(i,i), ZDrive=None, PFSOffset=None) for i in range(5)}
rand = randomise_fov(grid, 3)
assert isinstance(rand, dict) and len(rand) == 3
print("✅ randomise_fov passed. sample points:", list(rand.values()))

2025-04-24 15:43:59,671 [INFO] Temperature 2.562741203051934e-15. Current value: 8.485281374238571 k: 1/30 k_accepted: 1/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.562741203051934e-15. Current value: 8.48528137423857 k: 2/30 k_accepted: 2/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.562741203051934e-15. Current value: 8.485281374238571 k: 3/30 k_accepted: 3/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.3064670827467406e-15. Current value: 8.48528137423857 k: 1/30 k_accepted: 1/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.3064670827467406e-15. Current value: 8.48528137423857 k: 2/30 k_accepted: 1/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.3064670827467406e-15. Current value: 8.485281374238571 k: 3/30 k_accepted: 2/3 k_noimprovements: 0
2025-04-24 15:43:59,671 [INFO] Temperature 2.3064670827467406e-15. Current value: 8.48528137423857 k: 4/30 k_accepted: 3/3 k_noimprovements: 0

✅ randomise_fov passed. sample points: [StageCoord(xy=(0, 0), ZDrive=None, PFSOffset=None), StageCoord(xy=(3, 3), ZDrive=None, PFSOffset=None), StageCoord(xy=(2, 2), ZDrive=None, PFSOffset=None)]
